#### Config

In [7]:
import torch
import torchaudio
from einops import rearrange
from stable_audio_tools import get_pretrained_model
from stable_audio_tools.inference.generation import generate_diffusion_cond

device = "cuda" if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else "cpu"
print("Using {}".format(device))

Using mps


### General Stable Audio Open Model

Choose a model to download
Remember to get your HF Token and to register yourself to use the model on HF

[Stable Audio Open](https://huggingface.co/stabilityai/stable-audio-open-1.0)

[Stable Audio Open Small](https://huggingface.co/stabilityai/stable-audio-open-small)

In [2]:
# Download model | Stable Audio Open (normal)
# `https://huggingface.co/stabilityai/stable-audio-open-1.0`
model, model_config = get_pretrained_model("stabilityai/stable-audio-open-1.0")
sample_rate = model_config["sample_rate"]
sample_size = model_config["sample_size"]

model = model.to(device)

No module named 'flash_attn'
flash_attn not installed, disabling Flash Attention


/Users/marcocassar/Projects/DLAIE/self/sao/.venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [8]:
# Download model | Stable Audio Open Small
# `https://huggingface.co/stabilityai/stable-audio-open-small`
model, model_config = get_pretrained_model("stabilityai/stable-audio-open-small")
sample_rate = model_config["sample_rate"]
sample_size = model_config["sample_size"]

model = model.to(device)

/Users/marcocassar/Projects/DLAIE/self/sao/.venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [14]:
# Set up text and timing conditioning
conditioning = [{
    "prompt": "60 BPM techno beat",  # This prompt is quite bad on small, but small does work
    "seconds_start": 0,
    "seconds_total": 11
}]

# Generate stereo audio
output = generate_diffusion_cond(
    model,
    steps=100,
    cfg_scale=1.0,
    conditioning=conditioning,
    sample_size=sample_size,
    # sigma_min=0.3,
    # sigma_max=500,
    # sampler_type="dpmpp-3m-sde",  # Use this for normal open
    sampler_type="pingpong",  # Use this for small
    device=device
)


# Rearrange audio batch to a single sequence
output = rearrange(output, "b d n -> d (b n)")

# Peak normalize, clip, convert to int16, and save to file
output = output.to(torch.float32).div(torch.max(torch.abs(output))).clamp(-1, 1).mul(32767).to(torch.int16).cpu()

3068666184


100%|██████████| 100/100 [00:02<00:00, 35.23it/s]


In [15]:
# output: (channels, samples) float32 on CPU, normalized safely
from IPython.display import Audio, display
display(Audio(output.numpy(), rate=sample_rate))

In [16]:
# If you are having trouble running torch.save it's a known issue with torchcodec and mac
# It is caused by either a differing directory install of ffmpeg from Homebrew compared to conda or just a missing dependency of ffmpeg
# I provided a patch_torchcodec_mac.sh script that will clone, edit, and install ffmpeg
# It will then update torchcodec to point to the installed ffmpeg dir
# This is mainly a mac issue I think
#
# Run the script by doing ./patch_torchcodec_mac.sh
torchaudio.save("small_test.wav", output, sample_rate)